# Valorant Agent Picker Training

In this notebook, we wiil be using our 'Valorant Agent Picker' dataset to train a 'Random Forest Classifier' model and test its accuracy.

## Importing Dependencies

In [1]:
!pip install -q openpyxl


[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pickle

## Dataset Retreival

In [3]:
# GETTING THE DATASET FROM MY GITHUB REPOSITORY
!git clone https://github.com/omkanekar28/Valorant-Agent-Picker.git

Cloning into 'Valorant-Agent-Picker'...


In [4]:
dataset = pd.read_excel("Valorant-Agent-Picker/data/dataset_complete.xlsx")
dataset = dataset.sample(frac=1).reset_index(drop=True)
dataset

,Agent_Type,Playstyle,Difficulty,Ability_Preference,Gun_Type,Agent
0,Controller,Supportive,Easy,Agility,Snipers,Brimstone
1,Duelist,Info-gathering,Easy,Smokes,Machine Guns,Jett
2,Duelist,Aggressive,Hard,Agility,Rifles,Jett
3,Duelist,Supportive,Easy,Flashes/Stuns,Shotguns,Neon
4,Controller,Balanced,Hard,Smokes,Snipers,Viper
...,...,...,...,...,...,...
995,Initiator,Supportive,Easy,Healing,Shotguns,Skye
996,Sentinel,Map-control,Hard,Flashes/Stuns,Snipers,Vyse
997,Sentinel,Balanced,Easy,Flashes/Stuns,Snipers,Deadlock
998,Duelist,Supportive,Easy,Healing,SMGs,Iso


## Dataset Preprocessing

In [5]:
X = dataset.drop(columns=['Agent'])
y = dataset['Agent']

In [6]:
def preprocess_independent_variables(X):
  preprocessed_X = pd.get_dummies(X, columns=['Agent_Type', 'Playstyle', 'Ability_Preference', 'Gun_Type'], drop_first=True)    # ONE HOT ENCODING
  ordinal_encoder = OrdinalEncoder(categories=[['Easy', 'Hard']])
  preprocessed_X['Difficulty'] = ordinal_encoder.fit_transform(preprocessed_X[['Difficulty']])    # ORDINAL ENCODING
  return preprocessed_X

def preprocess_dependent_variable(y):
  label_encoder = LabelEncoder()
  preprocessed_y = label_encoder.fit_transform(y)
  class_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
  print(class_mapping)
  return preprocessed_y

In [7]:
X

,Agent_Type,Playstyle,Difficulty,Ability_Preference,Gun_Type
0,Controller,Supportive,Easy,Agility,Snipers
1,Duelist,Info-gathering,Easy,Smokes,Machine Guns
2,Duelist,Aggressive,Hard,Agility,Rifles
3,Duelist,Supportive,Easy,Flashes/Stuns,Shotguns
4,Controller,Balanced,Hard,Smokes,Snipers
...,...,...,...,...,...
995,Initiator,Supportive,Easy,Healing,Shotguns
996,Sentinel,Map-control,Hard,Flashes/Stuns,Snipers
997,Sentinel,Balanced,Easy,Flashes/Stuns,Snipers
998,Duelist,Supportive,Easy,Healing,SMGs


In [8]:
preprocessed_X = preprocess_independent_variables(X)
preprocessed_X

,Difficulty,Agent_Type_Duelist,Agent_Type_Initiator,Agent_Type_Sentinel,Playstyle_Balanced,Playstyle_Info-gathering,Playstyle_Map-control,Playstyle_Supportive,Ability_Preference_Flashes/Stuns,Ability_Preference_Healing,Ability_Preference_Information,Ability_Preference_Smokes,Gun_Type_Rifles,Gun_Type_SMGs,Gun_Type_Shotguns,Gun_Type_Snipers
0,0.0,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True
1,0.0,True,False,False,False,True,False,False,False,False,False,True,False,False,False,False
2,1.0,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False
3,0.0,True,False,False,False,False,False,True,True,False,False,False,False,False,True,False
4,1.0,False,False,False,True,False,False,False,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,False,True,False,False,False,False,True,False,True,False,False,False,False,True,False
996,1.0,False,False,True,False,False,True,False,True,False,False,False,False,False,False,True
997,0.0,False,False,True,True,False,False,False,True,False,False,False,False,False,False,True
998,0.0,True,False,False,False,False,False,True,False,True,False,False,False,True,False,False


In [9]:
y

0      Brimstone
1           Jett
2           Jett
3           Neon
4          Viper
         ...    
995         Skye
996         Vyse
997     Deadlock
998          Iso
999         Omen
Name: Agent, Length: 1000, dtype: object

In [10]:
preprocessed_y = preprocess_dependent_variable(y)
preprocessed_y

{'Astra': 0, 'Breach': 1, 'Brimstone': 2, 'Chamber': 3, 'Clove': 4, 'Cypher': 5, 'Deadlock': 6, 'Fade': 7, 'Gekko': 8, 'Harbor': 9, 'Iso': 10, 'Jett': 11, 'KAY/O': 12, 'Killjoy': 13, 'Neon': 14, 'Omen': 15, 'Phoenix': 16, 'Raze': 17, 'Reyna': 18, 'Sage': 19, 'Skye': 20, 'Sova': 21, 'Viper': 22, 'Vyse': 23, 'Yoru': 24}


array([ 2, 11, 11, 14, 22, 14,  5,  1, 13, 16, 19,  2, 15,  3, 21, 23,  8,
       20, 21,  8, 24, 19, 19,  5, 21, 11,  1, 21, 17, 23, 11,  7, 15,  5,
        8, 21, 24, 20,  1,  7,  9, 21, 19, 18, 10,  5, 19,  4, 15, 18,  7,
        4,  2, 19, 19,  9,  6, 11, 15, 15,  4,  1, 24, 15, 23,  9, 13, 14,
       18, 17,  4, 15, 24, 20,  3, 17, 16,  7, 21,  3, 20, 15,  7, 15,  7,
       18, 13,  3, 22,  7,  6, 15,  5, 16, 18, 20,  8,  0,  9,  1,  1,  2,
       24,  7, 13, 11, 22,  9, 20, 20,  9, 19, 23, 17, 21, 19, 19,  9, 11,
       16, 20, 23, 21,  0, 15, 18,  8,  2,  2, 18,  9, 23,  9, 14, 15, 16,
        5, 22, 16, 19, 16, 24,  6, 13, 16, 14, 19, 22,  4,  9,  9, 20, 11,
        9, 16, 14, 23, 22, 21, 17, 14,  6, 20,  5, 17, 20, 15, 14, 23, 15,
       19,  3, 15, 14,  8, 21,  9,  3, 15, 21,  5,  9,  9,  9,  1,  8, 20,
       19,  6,  8,  5, 13, 11,  6, 21,  7, 24,  9, 16, 19, 18,  7, 18,  8,
       13,  7, 21, 21,  9,  4,  2, 14, 20, 20,  3, 13,  4,  2, 17, 20, 23,
       13, 16, 15, 19, 15

## Train-Test Split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(preprocessed_X, preprocessed_y, test_size = 0.2, random_state = 0)

In [12]:
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

800
800
200
200


## Model Initialization and Training

In [13]:
classifier = RandomForestClassifier(
    n_estimators=500,
    criterion='entropy',
    random_state=42
)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=500, random_state=42)

## Model Evaluation

In [14]:
y_pred = classifier.predict(X_test)

In [15]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.91


In [16]:
report = classification_report(y_test, y_pred)
print("\nClassification Report:")
print(report)


Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00         4
           4       1.00      1.00      1.00         8
           5       0.80      0.80      0.80         5
           6       1.00      1.00      1.00         4
           7       0.86      1.00      0.92        12
           8       0.90      0.82      0.86        11
           9       1.00      1.00      1.00        11
          10       0.67      0.80      0.73         5
          11       1.00      0.80      0.89        10
          12       1.00      1.00      1.00         3
          13       0.90      0.90      0.90        10
          14       0.71      0.83      0.77         6
          15       1.00      1.00      1.00        13
          16       0.71      0.71      0.71         7
   

In [17]:
confusion = confusion_matrix(y_test, y_pred)
agents = y.unique().tolist()
confusion_with_labels = pd.DataFrame(confusion, index=agents, columns=agents)

# Adjust Pandas settings for full display
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.width', None)  # Adjust width to avoid line breaks
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping

print("Confusion Matrix:")
print()
confusion_with_labels

Confusion Matrix:



,Brimstone,Jett,Neon,Viper,Cypher,Breach,Killjoy,Phoenix,Sage,Omen,Chamber,Sova,Vyse,Gekko,Skye,Yoru,Raze,Fade,Harbor,Reyna,Iso,Clove,Deadlock,Astra,KAY/O
Brimstone,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Jett,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Neon,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Viper,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Cypher,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Breach,0,0,0,0,0,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
Killjoy,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Phoenix,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Sage,0,0,0,0,0,0,0,1,9,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
Omen,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Saving the Model

In [18]:
MODEL_STORE_PATH = f"RandomForestClassifier_ValorantAgentPicker_accuracy_{int(accuracy*100)}.pkl"

In [19]:
with open(MODEL_STORE_PATH, 'wb') as file:
  pickle.dump(classifier, file)

## Testing the Model (Inference)

In [20]:
# GLOBALS.PY
PREPROCESSED_X_COLUMNS = [
    'Difficulty',
    'Agent_Type_Duelist',
    'Agent_Type_Initiator',
    'Agent_Type_Sentinel',
    'Playstyle_Balanced',
    'Playstyle_Info-gathering',
    'Playstyle_Map-control',
    'Playstyle_Supportive',
    'Ability_Preference_Flashes/Stuns',
    'Ability_Preference_Healing',
    'Ability_Preference_Information',
    'Ability_Preference_Smokes',
    'Gun_Type_Rifles',
    'Gun_Type_SMGs',
    'Gun_Type_Shotguns',
    'Gun_Type_Snipers'
]

DIFFICULTY_MAPPINGS = {
    'Easy': 0.0,
    'Hard': 1.0
}

AGENTS = ['Phoenix', 'Reyna', 'Jett', 'Raze', 'Yoru', 'Neon', 'Iso', 'Sova', 
          'Breach', 'Skye', r'KAY/O', 'Fade', 'Gekko', 'Clove', 'Brimstone', 'Viper', 
          'Omen', 'Astra', 'Harbor', 'Vyse', 'Sage', 'Cypher', 'Killjoy', 
          'Chamber', 'Deadlock']


In [21]:
def get_preprocessed_input(input_dict):
  example_data_preprocessed = pd.DataFrame([[''] * len(PREPROCESSED_X_COLUMNS)], columns=PREPROCESSED_X_COLUMNS)
  example_data_columns = PREPROCESSED_X_COLUMNS[:]

  for column in example_data_columns:
    for current_column, current_column_value in input_dict.items():
      if current_column in column:
        # ORDINAL COLUMNS
        if current_column == 'Difficulty':
          example_data_preprocessed[column] = DIFFICULTY_MAPPINGS[input_dict['Difficulty']]
        # ONE HOT ENCODING COLUMNS
        else:
          if current_column_value in column:
            example_data_preprocessed[column] = True
          else:
            example_data_preprocessed[column] = False
  return example_data_preprocessed

In [22]:
example_data = {
    'Agent_Type': 'Initiator',
    'Playstyle': 'Info-gathering',
    'Difficulty': 'Easy',
    'Ability_Preference': 'Flashes/Stuns',
    'Gun_Type': 'Machine Guns'
}

example_data_preprocessed = get_preprocessed_input(example_data)

with open(MODEL_STORE_PATH, 'rb') as file:
  classifier = pickle.load(file)

example_data_pred = classifier.predict(example_data_preprocessed)
print(sorted(AGENTS)[example_data_pred[0]])

KAY/O
